In [21]:
import Bio
import numpy as np
from Bio import SeqIO
import sys
import csv
import os
import random as random
from functions_collection_copy import *

score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5

In [58]:
def make_related_strings(nr_of_str:int,len_of_str, degree_of_variation:float, start_string):
    string_fam_collection=[]
    first_string=[]
    if start_string==False:
        for k in range(0,int(len_of_str)):
                first_string.append(random.choice(['a','c','t','g']))
        first_string=''.join(first_string)
        string_fam_collection.append(first_string)
    else:
         first_string=start_string
         string_fam_collection.append(first_string)

    for i in range(1,nr_of_str):
        a_sequence=[]
        for element in first_string:
            if random.random() < degree_of_variation: #checking if we should change the charachter
                a_sequence.append(random.choice(['a','c','t','g']))
            else:
                 a_sequence.append(element)
        a_sequence=''.join(a_sequence)
        string_fam_collection.append(a_sequence)
    return string_fam_collection

#make_related_strings(3,12,0.1,False)


def make_strings_in_families(nr_of_fams, nr_str_pr_fam, len_of_str, internal_var_in_fams, degree_of_var_from_first_fam):
    all_strings_coll = []
    first_family = make_related_strings(nr_str_pr_fam, len_of_str, internal_var_in_fams, start_string=False)
    all_strings_coll.append(first_family)
    
    for i in range(1, int(nr_of_fams)):
        carry_over_string = first_family[0]
        mutated_carry_over = []
        for element in carry_over_string:
            if random.random() < degree_of_var_from_first_fam:  # checking if we should change the character
                mutated_carry_over.append(random.choice(['a', 'c', 't', 'g']))
            else:
                mutated_carry_over.append(element)
        mutated_carry_over = ''.join(mutated_carry_over)  # Join the list of characters into a string
        family = make_related_strings(nr_str_pr_fam, len_of_str, internal_var_in_fams,mutated_carry_over)
        all_strings_coll.append(family)
    all_strings_coll = [item for sublist in all_strings_coll for item in sublist]
    #all_strings_coll = [''.join(sublist) for sublist in all_strings_coll]
    return all_strings_coll


In [47]:
seqs=['aaaatc','agagtat','aatcctggta']
new_assembly_gradual_x(seqs,score_matrix,gap_cost,True)







integrity check 1 passed for seq 0: string same as original sequence
integrity check 1 passed for seq 1: string same as original sequence
integrity check 1 passed for seq 2: string same as original sequence
integrity test 2 passed for: 0 and 1: alignment cost consistent with guide alignment
integrity test 2 passed for: 0 and 2: alignment cost consistent with guide alignment
integrity test 3 passed: No empty columns in alignment


(array([[ 0., 11., 29.],
        [11.,  0., 29.],
        [29., 29.,  0.]]),
 [[['a', 'a', 'a'],
   ['a', 'g', 'a'],
   ['-', '-', 't'],
   ['-', '-', 'c'],
   ['-', '-', 'c'],
   ['-', '-', 't'],
   ['a', 'a', 'g'],
   ['a', 'g', 'g'],
   ['t', 't', 't'],
   ['-', 'a', '-'],
   ['c', 't', 'a']]],
 74,
 {'0': [0, 0], '1': [0, 1], '2': [0, 2]})

In [68]:
def test_many_parameters(nr_of_str=10, repeats=20,len_str=20,nr_of_fams=2, str_pr_fam=5, internal_var=0.1, var_between_fams=0.9,mode='random' ):
    costs_Krus=[]
    costs_Prim=[]
    costs_Gus=[]
    if mode=='random':
        random_sequences_collection=[]
        for i in range(0,repeats):
            random_pile=[]
            for j in range (0,nr_of_str):
                a_sequence=[]
                for k in range(0,len_str):
                    a_sequence.append(random.choice(['a','c','t','g']))
                a_sequence=''.join(a_sequence)
                random_pile.append(a_sequence)
            random_sequences_collection.append(random_pile)
        for random_pile in random_sequences_collection:
            #print('rand: I intend to align: '+str(random_pile))
            cost=new_assembly_gradual_x(random_pile,score_matrix,gap_cost)[2]
            costs_Krus.append(cost)
        for random_pile in random_sequences_collection:
            cost=new_assembly_Prim_x(random_pile,score_matrix,gap_cost)[2]
            costs_Prim.append(cost)
        for random_pile in random_sequences_collection:
            #print('rand:I intend to align: '+str(random_pile))
            cost=new_assembly_Gus_x(random_pile,score_matrix,gap_cost)[2]
            costs_Gus.append(cost)
    elif mode=='family':
        for i in range(0,repeats):
            #internal_var=[internal_var]*nr_of_fams
            #var_between_fams=[var_between_fams]*nr_of_fams
            strings_in_families=make_strings_in_families(nr_of_fams,str_pr_fam,len_str,internal_var,var_between_fams)
            #print('fam:I intend to align: '+str(strings_in_families))
            cost=new_assembly_gradual_x(strings_in_families,score_matrix,gap_cost)[2]

            costs_Krus.append(cost)

            cost=new_assembly_Prim_x(strings_in_families,score_matrix,gap_cost)[2]
            costs_Prim.append(cost)
          
            cost=new_assembly_Gus_x(strings_in_families,score_matrix,gap_cost)[2]
            costs_Gus.append(cost)
        
    else:
        print("that's not a valid mode, try 'family' or 'random'!")
    #result=costs_new_gradual, costs_new_one_by_one, costs_old
    result=costs_Krus,costs_Prim,costs_Gus
    if mode=='random':
        params="nr str: "+ str(nr_of_str)+", str len: "+str(len_str)+",mode:"+str(mode)
    if mode=='family':
        params="nr str: NA, str len: "+str(len_str)+", nr fams: "+str(nr_of_fams)+", str_pr_fam: "+str(str_pr_fam)+", internal var: "+ str(internal_var)+", var between fams: "+str(var_between_fams)+", mode:"+str(mode)
    return params, result

params,results=test_many_parameters(repeats=2,len_str=15,nr_of_fams=3,str_pr_fam=3,internal_var=0.1,var_between_fams=0.9, mode='family')

    
        

fam:I intend to align: ['aaaaatgctattcgc', 'aaaaatgcgactcgg', 'caaaatgctattcgc', 'tctagccccagttga', 'tctagccccagttaa', 'tctagctccagttga', 'ggatctgcccgattg', 'ggatctgcccgattg', 'cgatctgcccgattg']
























here5


























fam:I intend to align: ['agggttgcttcgaag', 'agggttggttcgaag', 'agggttgcttcgaaa', 'gtgcagtctgacaac', 'gtgcagtctgacacc', 'gtacagtctgtcaac', 'cgcaaacggctggaa', 'cgcaaacggcctgaa', 'cgcaaacggcttgaa']
























here5
























([1196, 1143], [1196, 1143], [1276, 1238])
nr str: NA, str len: 15, nr fams: 3, str_pr_fam: 3, internal var: 0.1, var between fams: 0.9, mode:family


In [73]:
def test_many_parameters_no_internal_reps(nr_of_str=10,len_str=20,nr_of_fams=2, str_pr_fam=5, internal_var=0.1, var_between_fams=0.9,mode='random' ):
    costs_Krus=[]
    costs_Prim=[]
    costs_Gus=[]
    if mode=='random':
        random_sequences_collection=[]
        random_pile=[]
        for j in range (0,nr_of_str):
            a_sequence=[]
            for k in range(0,len_str):
                a_sequence.append(random.choice(['a','c','t','g']))
            a_sequence=''.join(a_sequence)
            random_pile.append(a_sequence)
        random_sequences_collection.append(random_pile)
        for random_pile in random_sequences_collection:
            #print('rand: I intend to align: '+str(random_pile))
            cost=new_assembly_gradual_x(random_pile,score_matrix,gap_cost)[2]
            costs_Krus.append(cost)
        for random_pile in random_sequences_collection:
            cost=new_assembly_Prim_x(random_pile,score_matrix,gap_cost)[2]
            costs_Prim.append(cost)
        for random_pile in random_sequences_collection:
            #print('rand:I intend to align: '+str(random_pile))
            cost=new_assembly_Gus_x(random_pile,score_matrix,gap_cost)[2]
            costs_Gus.append(cost)
    elif mode=='family':
        #internal_var=[internal_var]*nr_of_fams
        #var_between_fams=[var_between_fams]*nr_of_fams
        strings_in_families=make_strings_in_families(nr_of_fams,str_pr_fam,len_str,internal_var,var_between_fams)
        #print('fam:I intend to align: '+str(strings_in_families))
        cost=new_assembly_gradual_x(strings_in_families,score_matrix,gap_cost)[2]

        costs_Krus.append(cost)

        cost=new_assembly_Prim_x(strings_in_families,score_matrix,gap_cost)[2]
        costs_Prim.append(cost)
        
        cost=new_assembly_Gus_x(strings_in_families,score_matrix,gap_cost)[2]
        costs_Gus.append(cost)
        
    else:
        print("that's not a valid mode, try 'family' or 'random'!")
    #result=costs_new_gradual, costs_new_one_by_one, costs_old
    result=costs_Krus,costs_Prim,costs_Gus
    if mode=='random':
        params="nr str: "+ str(nr_of_str)+", str len: "+str(len_str)+",mode:"+str(mode)
    if mode=='family':
        params="str len: "+str(len_str)+", nr fams: "+str(nr_of_fams)+", str_pr_fam: "+str(str_pr_fam)+", internal var: "+ str(internal_var)+", var between fams: "+str(var_between_fams)+", mode:"+str(mode)
    return params, result

params,results=test_many_parameters_no_internal_reps(len_str=15,nr_of_fams=3,str_pr_fam=3,internal_var=0.1,var_between_fams=0.9, mode='random')

    
        

In [77]:
#for random
repeats=5
mode='random'
str_len=[10,100]
nr_str=[3,5,10]

for i in range(0,repeats):
    for lens in str_len:
        for nr in nr_str:

            params,results=test_many_parameters_no_internal_reps(nr,lens,mode='random')

            csv_file_path="testing_outputs/random_strs_tests.csv"
            with open(csv_file_path, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(["Krus,Prim, Gus"])
                csvwriter.writerow([params,results])

In [78]:
#for families
repeats=5
mode='family'
nr_fams=[1,2,4]
str_pr_fam=[1,2,4,10]
internal_var=[0.1,0.5,0.9]
var_between_fams=[0.1,0.5,0.9]
str_len=[5,20]

csv_file_path="testing_outputs/familiy_outputs_testing.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow(["nr_of_str","str_len","mode","Kruskal","Prim","Gusfield"])

for i in range(0,repeats):
    for fam_num in nr_fams:
        for str_pr_fam_nr in str_pr_fam:
            for int_var in internal_var:
                for ex_var in var_between_fams:
                    for str_l in str_len:

                        params,results=test_many_parameters_no_internal_reps(len_str=str_l,nr_of_fams=fam_num,internal_var=int_var,var_between_fams=ex_var,mode=mode)

                        with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow([params,results])

KeyboardInterrupt: 